# Zero offset correction: calibrate pressure sensor

## Load and inspect data
Load pickle file and inspect contents

In [ ]:
import os
import pickle

# Import necessary pyologger utilities
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import *
from pyologger.process_data.sampling import upsample
from pyologger.calibrate_data.zoc import *
from pyologger.plot_data.plotter import plot_depth_correction

# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
dep_db = metadata.get_metadata("dep_DB")
logger_db = metadata.get_metadata("logger_DB")
rec_db = metadata.get_metadata("rec_DB")
animal_db = metadata.get_metadata("animal_DB")

# Assuming you have the metadata and dep_db loaded:
datareader = DataReader()
deployment_folder = datareader.check_deployment_folder(dep_db, data_dir)

if deployment_folder:
    datareader.read_files(metadata, save_csv=True, save_parq=True)

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

## Find dives
Involves a zero offset correction with `zoc`

In [ ]:
?smooth_downsample_derivative

In [ ]:
# Load the depth and temperature data
depth_data = data_pkl.data['CC-96']['depth'].values
temp_data = data_pkl.data['CC-96'].get('tempIMU')
sampling_rate = int(data_pkl.info['CC-96']['datetime_metadata']['fs'])
datetime_data = pd.to_datetime(data_pkl.data['CC-96']['datetime'])

# Sidebar for parameters
threshold = 0.1 # meters
min_duration = 30 # seconds
depth_threshold = 5 # meters
apply_temp_correction = False

# Dive detection parameters
min_depth_threshold = 1.0
dive_duration_threshold = 10
smoothing_window = value=5

# Process depth data
first_derivative, downsampled_depth = smooth_downsample_derivative(
    depth_data, 
    original_sampling_rate=sampling_rate, 
    downsampled_sampling_rate=1)

# Detect flat chunks (potential surface intervals)
flat_chunks = detect_flat_chunks(
    depth=downsampled_depth, 
    datetime_data=datetime_data[::int(sampling_rate)],  # Adjust datetime data to match downsampled depth
    first_derivative=first_derivative, 
    threshold=threshold, 
    min_duration=min_duration, 
    depth_threshold=depth_threshold, 
    original_sampling_rate=400, 
    downsampled_sampling_rate=1
)
num_flat_chunks = len(flat_chunks)
print(f"Number of potential surface intervals detected: {num_flat_chunks}")

# Apply zero offset correction
corrected_depth_temp, corrected_depth_no_temp, depth_correction = apply_zero_offset_correction(
    depth=downsampled_depth, 
    temp=temp_data.values if temp_data is not None else None, 
    flat_chunks=flat_chunks
)

# Upsample and adjust the corrected depths to match original sampling rate
upsampling_factor = int(sampling_rate / 1)
repeated_corrected_depth_temp = upsample(corrected_depth_temp, upsampling_factor, len(depth_data))
repeated_corrected_depth_no_temp = upsample(corrected_depth_no_temp, upsampling_factor, len(depth_data))

# Detect dives in the corrected depth data
dives = find_dives(
    depth_series=repeated_corrected_depth_no_temp,
    datetime_data=datetime_data,
    min_depth_threshold=min_depth_threshold,
    sampling_rate=sampling_rate,
    duration_threshold=dive_duration_threshold,
    smoothing_window=smoothing_window
)
num_dives = len(dives)
print(f"Number of dives detected: {num_dives}")


### Plot data

In [ ]:
?plot_depth_correction

In [ ]:
# Plotting
dec_factor = int(upsampling_factor)

fig = plot_depth_correction(datetime_data, dec_factor, depth_data, first_derivative, 
                            repeated_corrected_depth_temp, repeated_corrected_depth_no_temp, 
                            depth_correction, dives, flat_chunks, temp_data, apply_temp_correction)
fig.show()


### Save data

In [ ]:
# Save the corrected depth back to the data structure
if apply_temp_correction:
    data_pkl.data['CC-96']['corrdepth'] = repeated_corrected_depth_temp
else:
    data_pkl.data['CC-96']['corrdepth'] = repeated_corrected_depth_no_temp

with open(pkl_path, 'wb') as file:
        pickle.dump(data_pkl, file)